In [1]:
import pennylane as qml
import torch
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane.operation import Tensor

In [2]:
from sklearn.datasets import load_iris
def data_load_and_process():
    iris = load_iris()
    X = iris.data
    X = X / 4 * np.pi
    Y = iris.target
    return X, Y

In [3]:
num_qubits = 4
num_classes = 3
dev = qml.device("default.qubit", wires = num_qubits)


from pennylane.templates import AngleEmbedding
from pennylane.templates.layers import StronglyEntanglingLayers

@qml.qnode(dev)
def circuit(weights, x):
    AngleEmbedding(x, wires = range(num_qubits))
    StronglyEntanglingLayers(weights, wires = range(num_qubits))
    return qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2))

In [4]:
def variational_classifier(params, x):
    weights = params[0]
    bias1 = params[1]
    bias2 = params[2]
    M1, M2 = circuit(weights, x)
    M1_bias, M2_bias = M1 + bias1, M2 + bias2
    prediction = 2 * M1_bias + M2_bias
    return prediction

In [12]:
def cost_fn(params, X, labels):
    num_labels = len(labels)
    loss = 0
    predictions = [variational_classifier(params, x) for x in X]
    for l,p in zip(labels, predictions):
        loss += l * np.log(np.abs(p))
    return -loss

In [6]:
def accuracy(labels, predictions):
    acc = 0
    for l,p in zip(labels, hard_predictions):
        if np.abs(l - p) < 1e-2:
            acc = acc + 1
    acc = acc / len(labels)
    return acc

In [14]:
batch_size = 20
num_layers = 4
steps = 100
np.random.seed(0)

In [17]:
def training(num_layers, steps):
    
    weights = 0.01 * np.random.randn(num_layers, num_qubits, 3)
    bias1 = 0.0
    bias2 = 0.0
    params = [weights, bias1, bias2]
    
    opt = qml.NesterovMomentumOptimizer(0.1)
    
    for i in range(steps):
        batch_index = np.random.choice(len(X), size = batch_size)
        
        X_batch = X[batch_index]
        Y_batch = Y[batch_index]
        params, cost_new = opt.step_and_cost(lambda p: cost_fn(params, X_batch, Y_batch), params)  
        
        if i % 10 == 0:
            print("iteration: ", i, " cost: ", cost_new)
        
    return params

In [18]:
X, Y = data_load_and_process()
params = training(num_layers, steps)

iteration:  0  cost:  81.01507371481308
iteration:  10  cost:  55.381757792360176
iteration:  20  cost:  45.21236157553546
iteration:  30  cost:  87.39123981513406
iteration:  40  cost:  55.88971467031096
iteration:  50  cost:  85.08859236916295
iteration:  60  cost:  73.21073436475585
iteration:  70  cost:  74.69776836443545
iteration:  80  cost:  58.323353966015745
iteration:  90  cost:  43.50795816998282


In [21]:
predictions = [variational_classifier(params, x) for x in X]
print(predictions)

[tensor(-0.37710499, requires_grad=True), tensor(-0.1809945, requires_grad=True), tensor(-0.31522379, requires_grad=True), tensor(-0.19417406, requires_grad=True), tensor(-0.40219837, requires_grad=True), tensor(-0.22026651, requires_grad=True), tensor(-0.34297876, requires_grad=True), tensor(-0.29275031, requires_grad=True), tensor(-0.13144686, requires_grad=True), tensor(-0.19499271, requires_grad=True), tensor(-0.35525609, requires_grad=True), tensor(-0.23673058, requires_grad=True), tensor(-0.18314893, requires_grad=True), tensor(-0.26558135, requires_grad=True), tensor(-0.58041443, requires_grad=True), tensor(-0.33080225, requires_grad=True), tensor(-0.49295578, requires_grad=True), tensor(-0.37121118, requires_grad=True), tensor(-0.22060177, requires_grad=True), tensor(-0.36186294, requires_grad=True), tensor(-0.17818872, requires_grad=True), tensor(-0.34225684, requires_grad=True), tensor(-0.62765005, requires_grad=True), tensor(-0.14998748, requires_grad=True), tensor(-0.060052

In [22]:
print(Y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
